In [1]:
import json 
import numpy as np
import joblib
from tensorflow.keras.models import load_model
import random 
import pandas as pd
from text_utils import tokenize_and_lemma

In [2]:
model = load_model("chatbot_model.keras")
vectorizer = joblib.load("vectorizer.joblib")
le = joblib.load("label_encoder.joblib")
intents = json.loads(open("intents.json").read())
df = pd.DataFrame(intents['intents'])

In [3]:
df.head()

,tag,patterns,responses
0,greetings,"[Hello, Hey, Hi, Good Day, Greetings]","[Hello!, Hey, What Can I do for you?]"
1,Acne,"[blackheads, pimples, face pimples, neck pimpl...",[According to your symptoms you might have Acn...
2,Attention Deficit Disorder (ADHD),"[impulsivity, hyperactivity, lack of focus, me...",[According to your symptoms you might have Att...
3,Allergies,"[sneezing, itchy eyes, coughing, watery eyes, ...",[According to your symptoms you might have All...
4,Alzheimers,"[forgetful, confusion with time, poor judgement]",[According to your symptoms you might have Alz...


In [4]:
def predict_class(text, error_threshold=0.25, top_k=3):
    X = vectorizer.transform([text]).toarray()
    probs = model.predict(X, verbose=0)[0]

    idx_probs = [(i, p) for i, p in enumerate(probs) if p >= error_threshold]
    idx_probs.sort(key= lambda x: x[1], reverse=True)
    if not idx_probs:
        top = int(np.argmax(probs))
        idx_probs = [(top, probs[top])]

    intents_ranked = [{
        "intent": le.inverse_transform([i])[0], 
        "probablity":float(p)
    } for i, p in idx_probs[:top_k]]
    return intents_ranked

In [5]:
def get_response(predicted, intents_json):
    tag = predicted[0]["intent"]
    for it in intents_json["intents"]:
        if it["tag"] == tag:
            return random.choice(it["responses"])

    return "I'm not sure I understood that. Could you rephrase?"

In [6]:
user_text = "persistent mouth ulcers"
pred = predict_class(user_text)
resp = get_response(pred, intents)
# print(pred)  # [{'intent': 'flu', 'probability': 0.93}, ...]
print(resp)

You have Head and neck cancer


In [7]:
import sys
print(sys.executable)

c:\Users\Ankit Regmi\anaconda3\envs\tf_notebook\python.exe


In [8]:
!"{sys.executable}" -m pip install SpeechRecognition

In [9]:
!"{sys.executable}" -m pip install pyttsx3

In [10]:
import speech_recognition as sr
import pyttsx3
import time

In [11]:
# if __name__ == '__main__':
#     print("Bot is Running")

#     recognizer = sr.Recognizer()
#     mic = sr.Microphone()

#     def say(text):
#         # Re-init each time in notebooks to avoid “silent after first run”
#         engine = pyttsx3.init('sapi5')   # explicit driver on Windows
#         engine.setProperty('rate', 175)
#         engine.setProperty('volume', 1.0)
#         engine.say(text)
#         engine.runAndWait()
#         engine.stop()  # ensure driver releases
#         del engine

#     say("Hello user, I am Bagley, your personal Talking Healthcare Chatbot.")

#     with mic as symptom:
#         print("Say Your Symptoms. The Bot is Listening")
#         say("You may tell me your symptoms now. I am listening")
#         try:
#             recognizer.adjust_for_ambient_noise(symptom, duration=0.2)
#             symp = recognizer.listen(symptom)
#             text = recognizer.recognize_google(symp)
#             say("You said {}".format(text))
#             say("Scanning our database for your symptom. Please wait.")

#             time.sleep(1)

#             # Calling the function by passing the voice inputted
#             # symptoms converted into string
#             calling_the_bot(text)
#         except sr.UnknownValueError:
#             say(
#                 "Sorry, Either your symptom is unclear to me or it is\
#                 not present in our database. Please Try Again.")
#             print(
#                 "Sorry, Either your symptom is unclear to me or it is\
#                 not present in our database. Please Try Again.")
#         finally:
#             say(
#                 "If you want to continue please say True otherwise say\
#                 False.")
#         with mic as ans:
#             recognizer.adjust_for_ambient_noise(ans, duration=0.2)
#             voice = recognizer.listen(ans)
#             final = recognizer.recognize_google(voice)

#         # with mic as ans:
#         #     recognizer.adjust_for_ambient_noise(ans, duration=0.2)
#         #     voice = recognizer.listen(ans)
#         #     final = recognizer.recognize_google(voice)

#         # if final.lower() == 'no' or final.lower() == 'please exit':
#         #     say("Thank You. Shutting Down now.")
#         #     print("Bot has been stopped by the user")
#         #     exit(0)

In [13]:
print("Bot is running")
recognizer = sr.Recognizer()
def say(text):
    engine = pyttsx3.init('sapi5')
    engine.setProperty('rate', 175)
    engine.setProperty('volume', 1.0)
    engine.say(text)
    engine.runAndWait()
    engine.stop()  # ensure driver releases
    del engine

def calling_the_bot(txt):
    global res
    predict = predict_class(txt)
    res = get_response(predict, intents)

    say("Found it. From our Database we found that" + res)
    # engine.say(res)
    # engine.runAndWait()
    print("Your Symptom was  : ", txt)
    print("Result found in our Database : ", res)

say("Hello user, I am Bagley, your personal Talking Healthcare Chatbot.")
print("done")
with sr.Microphone() as symptom:
    print("Say your symptoms. The Bot is listening")
    say("You may tell me your symptoms now. I am listening")
    recognizer.adjust_for_ambient_noise(symptom, duration=0.4)
    symp = recognizer.listen(symptom, timeout=3, phrase_time_limit=10)
    text = recognizer.recognize_google(symp)
    say("You said {}".format(text))
    say("Scanning our database for your symptom. Please wait.")

    time.sleep(1)

    calling_the_bot(text)


Bot is running
done
Say your symptoms. The Bot is listening


WaitTimeoutError: listening timed out while waiting for phrase to start